# Art Gallery Problem

## Geometrical Helper Classes

In [58]:
# Point: (x|y)
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f"{self.__class__.__name__}(x={self.x},y={self.y})"
    def __str__(self):
        return f"({self.x}|{self.y})"

In [59]:
# Line with equation: ax + by = c
class Line:
    def __init__(self,a,b,c):
        self.a = a
        self.b = b
        self.c = c
    def __repr__(self):
        return f"{self.__class__.__name__}(a={self.a},b={self.b},c={self.c})"
    def __str__(self):
        return f"{self.a}x + {self.b}y = {self.c}"

In [60]:
# Line segment from Point start to Point end
class LineSegment(Line):
    def __init__(self,start,end):
        self.start = start
        self.end = end
        
        
    def __repr__(self):
        return f"{self.__class__.__name__}(start={repr(self.start)},end={repr(self.end)})"
    def __str__(self):
        return f"{self.start} <-> {self.end}"